In [172]:

from dataclasses import dataclass
from enum import Enum


class State(Enum):
    NORMAL = 0
    BURNING = 1
    DEAD = 2

    def __str__(self) -> str:
        return self.name

    def get_letter(self):
        LETTERS = {
            self.BURNING: "B",
            self.DEAD: "D",
        }
        return LETTERS[self]

class Type(Enum):
    CHAPARRAL = 0
    CANYON = 1
    FOREST = 2
    LAKE = 3
    TOWN = 4

    def __str__(self) -> str:
        return self.name

    def get_fuel(self):
        FUEL = {
            self.CANYON: 1,
            self.CHAPARRAL: 28,
            self.FOREST: 120,
            self.LAKE: -1,
            self.TOWN: -1,
        }
        return FUEL[self]

    def burning_propability(self): #how easy to ignite relatives coeficients
        PROBS = {
            self.CANYON: 1,
            self.CHAPARRAL: 0.5,
            self.FOREST: 0.2,
            self.LAKE: 0,
            self.TOWN: 1,
        }
        return PROBS[self]

    def get_letter(self):
        LETTERS = {
            self.CANYON: "K",
            self.CHAPARRAL: "C",
            self.FOREST: "F",
            self.LAKE: "L",
            self.TOWN: "T",
        }
        return LETTERS[self]



@dataclass
class Cell(object):
    def __init__(self, type_int: int) -> None:
        self.type = Type(type_int)
        self.fuel = self.type.get_fuel()
        self.state = State.NORMAL

    def __str__(self):
        if self.state!= State.NORMAL:
            return self.state.get_letter()
        return self.type.get_letter()

    def update(self, ignite: bool = False):

        # decrease fuel when burning
        if self.is_burning:
            self.fuel -= 1

        # cell is dead if ran out of fuel
        if self.fuel == 0:
            self.state = State.DEAD

        # ignite the cell
        if ignite and self.can_ignite:
            self.state = State.BURNING

        return self.stop_condition
        
    @property
    def is_burning(self) -> bool:
        return self.state == State.BURNING
    
    @property
    def stop_condition(self) -> bool:
        return self.type == Type.TOWN and self.is_burning
    @property
    def can_ignite(self) -> bool:
        return (self.state == State.NORMAL and self.type != Type.LAKE)


In [173]:
import numpy as np
import random

In [174]:
class Board:
    def __init__(self, wind_vec):
        #set up
        init_arry = np.genfromtxt('the_grid.csv', delimiter=',')
        vCell = np.vectorize(Cell)
        self._matrix  = np.empty(init_arry.shape, dtype=object)
        self._matrix[:,:]   = vCell(init_arry)
        self.height, self.width = self._matrix.shape
        self.wind = self.normalize(wind_vec)

    def __str__(self):
        row_repr = ""
        for row in self._matrix:
            row_repr += " ".join([str(cell) for cell in row] + ["\n"])
        return row_repr

    def get_cell(self, y, x):
        return self._matrix[y][x]

    def normalize(self, v): #normalize vector
        norm = np.linalg.norm(v)
        if norm == 0: 
            return v
        return v / norm
    def get_wind_coeffient(self, spread_direction):
        bias = 1
        #do we allow come bias?
        return np.dot(self.wind, self.normalize(spread_direction)) + bias 

    def _get_neighbors_indices(self, y, x):
        indices = [
            (y+1,x-1), #nw
            (y+1,x), #n
            (y+1,x+1), #ne

            (y,x+1), #e

            (y-1,x+1), #se
            (y-1,x), #s
            (y-1,x-1), #sw

            (y,x-1), #w
        ]
        filtered_indices = [
            (y,x) for y,x in indices
                    #check if in grid boundaries 
                    if y in range(0, self.height) and x in range(0, self.width)
        ]
        return filtered_indices
        
    def _should_ignite_cell(self,y,x) ->bool:
        cell = self.get_cell(y,x)
        
        neighbors = self._get_neighbors_indices(y, x)
        
        BASE_IGNITION_PROP = 0.1
        #from neighbours
        total_ignition_prop = 0
        for neighbor_y, neighbor_x  in neighbors:
            neighbor = self.get_cell(neighbor_y,neighbor_x)
            if neighbor.is_burning:

                #WIND IMPACT
                #direction vector from neighbor to the cell
                spread_dir = np.array([neighbor_y-y, neighbor_x-x])

                #strongest when wind
                wind_coeff = self.get_wind_coeffient(spread_dir)
                print(wind_coeff)
                total_ignition_prop += BASE_IGNITION_PROP*wind_coeff

        #from cell type
        cell_type_ignition_coefficient = cell.type.burning_propability()
        total_ignition_prop *=cell_type_ignition_coefficient


        ignite = random.random() < total_ignition_prop
        return ignite
    
    def update(self):
        #transition function
        for y in range(self.height):
            for x in range(self.width):
                ignite = self._should_ignite_cell(y,x)
                game_over = self.get_cell(y,x).update(ignite)
                if game_over:
                    return game_over


In [175]:
from IPython.display import clear_output
from time import sleep
NORTH_WIND = np.array([-1,0])
b = Board(NORTH_WIND)

#ignite top left corner
b.get_cell(0,0).update(True)
# print(b)
clear_output(wait=True)

In [176]:
for i in range(1000):
    clear_output(wait=True)
    print(b)
    sleep(0.5)
    if (b.update()):
        clear_output(wait=True)
        print(b)
        print(f"Game over after {i} steps")
        break

D D D D D D B C B B B B C C C C C C C C C C C C C C C C C C C C C C C C C C C C 
D D D D D D D D D B B B B C C C C C C C C C C C C C C C C C C C C C C C C C C C 
D D D D D D D D D B B B B B C C C C C C C C C C C C C C C C C C C C C C C C C C 
D D D D D D D D D D B B B B C C C C C C C C C C C C C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B B F F F F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B F F F F F F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B F F F F F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B B B F F F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B F B B F F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B B B B B F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B B B B B F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B B B F F F F C C C C K K C C C C C C C C C C C C C C 
D D D D D D D D D D D D B B 